<a href="https://colab.research.google.com/github/kyawmoeaung/PyJuliaR/blob/main/field_delineation_S2_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Delineating Crop Field Boundaries from Sentinel-2 (S2) imagery with Segment-Anything Model (SAM)

**GPU runtime is suggested to run this script.**

This script delineates the crop field boundaries on S2 imagery by using the <i>segment-geospatial</i> Python package developed by Prof. Qiusheng Wu at the University of Tennessee, Knoxville. The <i>segment-geospatial</i> package utilizes Meta's Segment-Anything Model (SAM) to generate segmented masks of different features on a remote sensing imagery as separated polygons. Hence, theoretically, if SAM is applied over the croplands, the boundaries of the segmented polygons can be considered as the crop field boundaries.


This script attempts to answer two questions:    
1. If SAM can really be used to delineate the crop field boundaries?

2. Since false-color image enhance the contrast between crops that have different health conditions, can SAM reveal the boundaries between crops that have different health conditions if it is applied to false-color image?

To answer these two question, I retrieved both S2 true-color and false-color images from the Google Earth Engine, then delineated the crop field boundaries using SAM. The results were compared with the USDA Cropland Data Layer, as well as the coincident S2 NDVI image.

The results showed that the boundaries delineated from both types of imagery generally agree with the boundaries shown on the USDA Cropland Data Layer. This indicates that SAM can be used as a tool that quickly delineate the field boundaries.

In addition, the field boundaries delineated from the false-color image seem to better reveal some details shown on the NDVI image, indicating that applying SAM on the false-color image may be used to show the boundaries between crops that have different health conditions. However, more examination on the delineated results is required to reach a solid conclusion.





# Prepare the required packages

Install and import the packages required to run this script.

In [1]:
!pip install geemap rioxarray segment-geospatial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.6/565.6 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

import ee
import geemap
import google.colab.drive as drive

# Prepare S2 images to be used for crop delineation

## Get the Google Earth Engine API and Google Drive ready

Since we will be retrieving the S2 images from the Google Earth Engine, first of all we need to authenticate and initialize the Google Earth Engine API so that we can use it in this Colab script.

In [4]:
ee.Authenticate()
ee.Initialize(project="ee-pyjuliar")

Then, let's mount our Google Drive to the Google Colab for the convenience of data export and access.

In [5]:
drive.mount('/content/drive/')
root_folder = '/content/drive/MyDrive/'

Mounted at /content/drive/


## Define an Area-Of-Interest (AOI)

An AOI is where the retrieved S2 images will cover. We will be performing delineation for the crop fields inside the AOI as well.

In this script, I defined a small AOI in Iowa state, the state that has the highest average percentage of corn and soybean productions in the U.S.

In [6]:
# Define the AOI with a bounding box
sample_aoi_w, sample_aoi_s, sample_aoi_e, sample_aoi_n = -94., 43.25, -93.9, 43.35
sample_aoi = ee.Geometry.BBox(sample_aoi_w, sample_aoi_s, sample_aoi_e, sample_aoi_n)

# Visualize the AOI.
Map = geemap.Map(center=((sample_aoi_n+sample_aoi_s)/2,(sample_aoi_w+sample_aoi_e)/2), zoom=12)
Map.addLayer(sample_aoi,{},name='Sample AOI')
Map.addLayerControl()
Map

Map(center=[43.3, -93.95], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

## Retrieve S2 composite images from the Google Earth Engine

Now, let's go ahead and retrieve S2 true-color and false-color images from the Google Earth Engine.

In [7]:
def s2_composite(doi, sample_aoi, band_list):

    """
    This is a mapping function that retrieve S2 image collection of defined bands at
    the given acquisition time inside an AOI.
    """

    # A mapping function for cloud masking and value scaling
    def maskS2clouds(image):
        qa = image.select('QA60');

        # QA has 12 bits from bit-0 to bit-11
        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10; # Push "1" 10 spaces to the left (010000000000)
        cirrusBitMask = 1 << 11; # Push "1" 11 spaces to the left (100000000000)

        # "bitwiseAnd" compare the QA and "cloudBitMask" and "cirrusBitMask"
        # then return True if (1) bit-10 of QA and "cloudBitMask" do not match (no cloud)
        #                     (2) bit-11 of QA and "cirrusBitMask" do not match (no cirrus cloud)
        #
        # Both flags should be set to zero, indicating clear conditions.
        # (a pixel that is neither cloud nor cirrus cloud will be retained)
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
        #
        # Mask out the cloud/cirrus-cloud pixels and scale the data by 10000 (the scale factor of the data)
        return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])

    # A mapping function to scale the composited images to 0-255 range
    def rgb_uint8(image):
        return image.multiply(255).uint8().copyProperties(image, ['system:time_start'])

    # A mapping function to clip the image collection to the defined AOI
    def imgcol_clip(image):
        return image.clip(sample_aoi)


    doi_end = ee.Date(doi).advance(1,'day').format('YYYY-MM-dd')

    # Retrieve the selected bands of S2 image collection
    color_outfname = 's2_'+''.join(band_list)+'_'+doi
    s2_composite_ImgCol = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(doi, doi_end)
        ##Pre-filter to get less cloudy granules.
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
        .filterBounds(sample_aoi)
    ).map(maskS2clouds).select(band_list).map(rgb_uint8).map(imgcol_clip)

    return s2_composite_ImgCol, color_outfname


The true-color images are nothing special but just visualizing the Red, Green, and Blue band reflectance as Red, Green, and Blue as they are.

For the false-color images, I used the most common band combination, which visualizes Near-Infrared (NIR), Red, and Green band reflectances as the Red, Green, and Blue on the images, repsectively. This band combination better reveals the health condition of crops than the true color images. Since healthier crops have much stronger NIR reflectances which are visualized as red, healthier crops will appear in bright red on the false-color images. On the other hand, the less healthier crops or bare ground will appear in tan.

I arbitrarily selected a Date-Of-Interest (DOI) on 2022-07-13 to get S2 true-color and false-color images. According to USDA, July is the growing season of both corn and soybean (https://ipad.fas.usda.gov/countrysummary/Default.aspx?id=US&crop=Corn).



![](https://drive.google.com/uc?export=view&id=1-6gE5fC5Fa3nC7m-GJT0_5wrGjGq2RLR)


![](https://drive.google.com/uc?export=view&id=1-E3szEBN1Ox77OLrCWiHHGfJGv3LNssq)


Since there may have several tiles of image in the AOI on the DOI, I only used the earliest acquired image on the DOI here.

In [19]:
# Define DOI, and true-color, and false-color band lists.
doi = '2022-07-13'

true_color_bands = ['B4','B3','B2']
false_color_bands = ['B8','B4','B3']

# Retreive S2 true-color and false-color images from the Google Earth Engine
true_color_s2_ImgCol, true_color_s2_outfname = s2_composite(doi, sample_aoi, true_color_bands)
false_color_s2_ImgCol, false_color_s2_outfname = s2_composite(doi, sample_aoi, false_color_bands)

# Get the earliest-acquired image on the DOI
true_color_s2_Img = true_color_s2_ImgCol.first()
false_color_s2_Img = false_color_s2_ImgCol.first()

In [9]:
# Visualize retrieved S2 true-color and false-color images
Map_Img = geemap.Map(center=((sample_aoi_n+sample_aoi_s)/2,(sample_aoi_w+sample_aoi_e)/2), zoom=12)
Map_Img.addLayer(true_color_s2_Img, {'min':0,'max':255,'bands':true_color_bands, 'gamma':1.7}, name=true_color_s2_outfname)
Map_Img.addLayer(false_color_s2_Img, {'min':0,'max':255,'bands':false_color_bands, 'gamma':1.7}, name=false_color_s2_outfname)
Map_Img.addLayerControl()
Map_Img

Map(center=[43.3, -93.95], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

Since the <i>segment-geospatial</i> package use GeoTIFF as input, now let's export the S2 true-color and false-color images that we just retrieved as GeoTIFFs to our Google Drive.

In [10]:
def export_eeImg(ee_image, export_folder, export_filename, scale=10):
    """
    A function that exports an Earth Engine image to Google Drive.
    "scale" is the export spatial resolution in meter.
    """

    task = ee.batch.Export.image.toDrive(**{
        'image': ee_image,
        'description': export_filename,
        'folder':export_folder,
        'scale': scale,
        'region': ee_image.geometry().getInfo()['coordinates']
    })
    task.start()

In [11]:
# Define the folder in Google Drive that you want to export the image to
export_folder = 'sentinel2_composite'

# Export the image
export_eeImg(true_color_s2_Img, export_folder, true_color_s2_outfname, scale=10)
export_eeImg(false_color_s2_Img, export_folder, false_color_s2_outfname, scale=10)


# Crop field delineation from S2 images using SAM

Now that we have the S2 true-color and false-color images ready, let's delineate the crop field boundary.

## SAM segmentation

In [12]:
def sam_segment(input_image_path, output_folder):

    """
    A wrap-up function that applying SAM to segment the crop field into polygons.
    The boundaries of polygons are considered as crop field boundaries
    """

    output_segment_path = output_folder+input_image_path.split('/')[-1][:-4]+'_delineation.tif'

    sam.generate(
        input_image_path, output_segment_path, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
    )

    output_segment_vector_path = output_segment_path[:-4]+".gpkg"
    output_segment_shp_path = output_segment_path[:-4]+".shp"
    sam.tiff_to_gpkg(output_segment_path, output_segment_vector_path, simplify_tolerance=None)
    sam.tiff_to_vector(output_segment_path, output_segment_shp_path)

    return output_segment_vector_path


In [13]:
# Initialize SAM modeol
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

Model checkpoint for vit_h not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:17<00:00, 145MB/s]


In [20]:
root_folder = "/content/drive/MyDrive/"
export_folder = "sentinel2_composite"

input_image_folder = root_folder+export_folder+"/"
sam_segment_outfolder = input_image_folder+"sam/"
if not os.path.exists(sam_segment_outfolder):
    os.makedirs(sam_segment_outfolder)


# If you run into error saying the image does not exist, please check if two images were
# exported to different folders in Google Drive. Earth Engine API creates a renamed folder if it is already exists.
true_color_image_path = input_image_folder+true_color_s2_outfname+".tif"
true_color_sam_vector_path = sam_segment(true_color_image_path, sam_segment_outfolder)

false_color_image_path = input_image_folder+false_color_s2_outfname+".tif"
false_color_sam_vector_path = sam_segment(false_color_image_path, sam_segment_outfolder)

ValueError: Input path /content/drive/MyDrive/sentinel2_composite/s2_B4B3B2_2022-07-13.tif does not exist.

## Visualize the delineated crop field boundaries

Now we have successfully segmented the S2 true-color and false-color images into polygons, whose boundaries can be considered as crop field boundaries. Let's visualize them!

Since I am also curious if using false-color images for crop field delineation can better differentiate crop fields with different health conditions, I also calculated the Normalized Difference Vegetation Index (NDVI) as reference for comparison.

Simply put, NDVI is the normalized difference of NIR and red band reflectances. Since healthy crops reflect lots of NIR and absorb more red light for photosynthesis process and creating chlorophyll, higher NDVI indicating healthier crops.

In [15]:
def s2_vi(doi, vi_type, sample_aoi):

    """
    A wrap-up function that retrieves and calculates S2 vegetaion indices (e.g., NDVI, LSWI)
    """

    def maskS2clouds(image):
        qa = image.select('QA60');

        # QA has 12 bits from bit-0 to bit-11
        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10; # Push "1" 10 spaces to the left (010000000000)
        cirrusBitMask = 1 << 11; # Push "1" 11 spaces to the left (100000000000)

        # "bitwiseAnd" compare the QA and "cloudBitMask" and "cirrusBitMask"
        # then return True if (1) bit-10 of QA and "cloudBitMask" do not match (no cloud)
        #                     (2) bit-11 of QA and "cirrusBitMask" do not match (no cirrus cloud)
        #
        # Both flags should be set to zero, indicating clear conditions.
        # (a pixel that is neither cloud nor cirrus cloud will be retained)
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
        #
        # Mask out the cloud/cirrus-cloud pixels and scale the data by 10000 (the scale factor of the data)
        return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])

    def imgcol_clip(image):
        return image.clip(sample_aoi)


    def cal_lswi(image):
        lswi = image.expression(
            "(NIR - SWIR)/(NIR + SWIR)",
            {
                "NIR": image.select('B8'),
                "SWIR": image.select('B11')
            }
        ).rename('LSWI').copyProperties(image, ['system:time_start']);
        #image = image.addBands(ndvi)

        return lswi


    def cal_ndvi(image):
        ndvi = image.expression(
            "(NIR - RED)/(NIR + RED)",
            {
                "NIR": image.select('B8'),
                "RED": image.select('B4')
            }
        ).rename('NDVI').copyProperties(image, ['system:time_start']);
        #image = image.addBands(ndvi)

        return ndvi

    doi_end = ee.Date(doi).advance(1,'day').format('YYYY-MM-dd')

    # Retrieve S2 images with selected bands
    color_outfname = 's2_'+vi_type+'_'+doi
    s2_ImgCol = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(doi, doi_end)
        ##Pre-filter to get less cloudy granules.
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
        .filterBounds(sample_aoi)
    ).map(maskS2clouds)

    if vi_type == 'lswi':
        vi_ImgCol = s2_ImgCol.map(cal_lswi).map(imgcol_clip)
    elif vi_type == 'ndvi':
        vi_ImgCol = s2_ImgCol.map(cal_ndvi).map(imgcol_clip)

    return vi_ImgCol, color_outfname

In [16]:
def add_img_vec(map_obj, ee_img, band_list, segment_vector, ee_img_name, style, gamma_value=2., plot_image_opt=False):

    """
    A wrap-up function for visualizing S2 images and the correpsonding crop field boundaries.
    """
    if plot_image_opt==True:
        map_obj.addLayer(ee_img, {'min':0,'max':255,'bands':band_list,'gamma':gamma_value}, name=ee_img_name)

    map_obj.add_vector(segment_vector, layer_name=ee_img_name+'_delineation', style=style)


In [17]:
# Define the DOI, it has to be the same as the crop field boundaries that we just
# delineated from S2 images
#doi = '2022-07-13'

# Calculate the S2 NDVI in the AOI on DOI
ndvi_s2_ImgCol, ndvi_s2_outfname = s2_vi(doi, 'ndvi', sample_aoi)

# Get the earliest-acquired image on DOI
ndvi_s2_Img = ndvi_s2_ImgCol.first()


In [18]:
# Style for visualizing the segmented polygons
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.,
}

# Retrieve USDA Cropland data as background
usda_crop = (
    ee.ImageCollection("USDA/NASS/CDL")
    .filterDate('2022-01-01','2022-12-31')
    .first()
    .select('cropland')
) .clip(sample_aoi)


Map_ImgSeg = geemap.Map(center=((sample_aoi_n+sample_aoi_s)/2,(sample_aoi_w+sample_aoi_e)/2), zoom=14)
Map_ImgSeg.addLayer(usda_crop,{},name='USDA NASS Cropland 2022')
Map_ImgSeg.addLayer(ndvi_s2_Img,{"min":0,"max":1},name='s2 NDVI_'+doi)
add_img_vec(Map_ImgSeg, true_color_s2_Img, true_color_bands, true_color_sam_vector_path, true_color_s2_outfname, style)
add_img_vec(Map_ImgSeg, false_color_s2_Img, false_color_bands, false_color_sam_vector_path, false_color_s2_outfname, style)
Map_ImgSeg.addLayer(ee.Geometry.BBox(-93.930, 43.287, -93.921, 43.299), {'color':'red'}, opacity = 0.5)
Map_ImgSeg.addLayer(ee.Geometry.BBox(-93.991, 43.308, -93.985, 43.312), {'color':'red'}, opacity = 0.5)
Map_ImgSeg.addLayer(ee.Geometry.BBox(-93.972, 43.29, -93.931, 43.30), {'color':'red'}, opacity = 0.5)
Map_ImgSeg.addLayerControl()
Map_ImgSeg

NameError: name 'true_color_sam_vector_path' is not defined

By comparing the crop field boundaries delineated by SAM from S2 true-color and false-color images, we can see that they both generally agree with the boundaries shown on the USDA cropland data. This indicates that SAM can be used as a tool that quickly delineate the field boundaries.

If we take a closer look, we can find that in some areas the field boundaries delineated from the false-color image better agree with the details shown on the NDVI image (e.g., the areas marked by red boxes).

This indicates that SAM may be used to reveal the boundaries between the crops that have different health conditions when being applied to the false-color image. However, more examination on the delineated boundaries is required to reach a solid conclusion.